In [1]:
from transformers import BertConfig, BertForMaskedLM, BertTokenizer
from transformers import PreTrainedTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torch.nn.functional import pad
from datasets import load_dataset
from tqdm.notebook import tqdm
from itertools import cycle
import wandb

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
wandb.login(key='56853ea1087c0d089c005f3f7aede52740245615')

device

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mb_lims (lims-ai). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/mburtsev/.netrc


device(type='cuda', index=0)

# Create a custom tokenizer for CA

In [2]:
class SimpleTokenizer:
    def __init__(self):
        self.token_to_id = {'0': 0, '1': 1, '[SEP]': 2, '[PAD]': 3, '[T]': 4} # [T] - thought
        self.id_to_token = {v: k for k, v in self.token_to_id.items()}

    def encode(self, text):
        return [self.token_to_id.get(token, self.token_to_id['[PAD]']) for token in text]

    def decode(self, tokens):
        return ''.join([self.id_to_token.get(token, '[PAD]') for token in tokens])
        #return ''.join([self.id_to_token[token] for token in tokens])

    def __len__(self):
        return len(self.token_to_id)

tokenizer = SimpleTokenizer()

# Define CA dataset

In [3]:
class CellularAutomataDataset(Dataset):
    
    def __init__(self, hf_dataset, steps, horizon=1, pad_length=512):
        self.hf_dataset = hf_dataset
        self.pad_length = pad_length
        self.steps = steps
        self.horizon = horizon

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        row = self.hf_dataset[idx]

        # Assuming the first element 'rule' is not used for model input
        # Concatenate all states except the last one for input
        if (self.steps > (len(row) - 1)):
            self.steps = len(row) - 1
        
        input_ids = []
        for t in range(self.steps):  # Exclude the last state and the 'rule' row
            input_ids.extend(tokenizer.encode(row[f't={t}']))
            input_ids.append(2)  # Append [SEP] token after each state

        # Use the last state as target and calculate its length
        target_ids = []
        for t in range(self.horizon):
            last_state_key = f't={self.steps + t}'
            target_ids.extend(tokenizer.encode(row[last_state_key]))
            target_ids.append(2)
        
        orbit_len = len(input_ids)
        target_len = len(target_ids)
        # Padding input and target sequences to the fixed length
        input_ids = pad(torch.tensor(input_ids), (0, self.pad_length - orbit_len), value=3)
        target_ids = pad(torch.tensor(target_ids), (orbit_len, self.pad_length - orbit_len - target_len), value=3)

        # Create a tensor of -100 and replace the last part with target_ids
       # target_labels = torch.full((self.pad_length,), -100)
        #target_labels[:state_length] = target_ids[:state_length]

        return input_ids, target_ids #target_labels


# Configuration for the BERT model

In [19]:
hidden_size=512
num_hidden_layers=10
num_attention_heads=8

config = BertConfig(
    vocab_size=len(tokenizer),
    hidden_size=hidden_size,
    num_hidden_layers=num_hidden_layers,
    num_attention_heads=num_attention_heads,
    #intermediate_size=3072
)
model_name = 'BERT'
model = BertForMaskedLM(config=config)

# Load dataset from Hugging Face repository

In [20]:
seed = 0
batch_size = 400
torch.manual_seed(seed)
steps = 10
state_len = 21 
look_ahead = 4
orbit_len = state_len * steps
seq_len = orbit_len + state_len * look_ahead

dataset_name = "1dCA_r2s20T20"
hf_dataset = load_dataset("mbur/"+dataset_name)['train']
dataset = CellularAutomataDataset(hf_dataset, steps, look_ahead, pad_length=seq_len)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

hf_dataset = load_dataset("mbur/"+dataset_name)['validation']
val_dataset = CellularAutomataDataset(hf_dataset, steps, look_ahead, pad_length=seq_len)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

hf_dataset = load_dataset("mbur/"+dataset_name)['test']
test_dataset = CellularAutomataDataset(hf_dataset, steps, look_ahead, pad_length=seq_len)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

validation_size = 1000  # Number of samples for validation
small_val_dataset, _ = random_split(val_dataset, [validation_size, len(val_dataset) - validation_size])

small_val_dataloader = DataLoader(small_val_dataset, batch_size=batch_size, shuffle=True)

Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


In [16]:
# Define the ID of the sample you want to inspect
sample_id = 1  # Replace with the ID of the sample you're interested in

# Fetch the specific sample from the dataset
sample_input_ids, sample_target_ids = dataset[sample_id]
sample_input_ids, sample_target_ids = sample_input_ids.unsqueeze(0).to(device), sample_target_ids.unsqueeze(0).to(device)


# Decode the tokens to strings for readability
decoded_input = tokenizer.decode(sample_input_ids[0].tolist())
decoded_target = tokenizer.decode(sample_target_ids[0].tolist())

# Print input, target, prediction, and logits
print("inputs:",len(sample_input_ids),sample_input_ids.tolist())
print("targets:",len(sample_target_ids),sample_target_ids.tolist())
print("Input:", len(sample_input_ids[0].tolist()),decoded_input)
print("Target:", len(sample_target_ids[0].tolist()), decoded_target)

inputs: 1 [[0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 2, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 2, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 2, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 2, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]]
targets: 1 [[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

# Evaluation

In [7]:
def evaluate_model(model, val_dataloader, device):
    model.eval()  # Set the model to evaluation mode
    total, correct = 0, 0
    val_loop = tqdm(val_dataloader, leave=False, desc="Validation")

    with torch.no_grad():
        for input_ids, target_ids in val_loop:
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            outputs = model(input_ids)
            predictions = torch.argmax(outputs.logits, dim=-1)

            # Create a mask where target_ids is not -100 (ignoring padding)
            valid_mask = target_ids != 3

            # Apply the mask to filter out -100 values
            valid_predictions = predictions[valid_mask]
            valid_targets = target_ids[valid_mask]

            correct += (valid_predictions == valid_targets).sum().item()
            total += valid_targets.numel()

    accuracy = correct / total if total > 0 else 0
    return accuracy

# Training loop

In [8]:
model.to(device)

lr=1e-4 # learning rate

run_cfg = {
    'dataset': dataset_name,
    'model': model_name,
    'input_size': 512,
    'hidden_size': hidden_size,
    'num_hidden_layers': num_hidden_layers,
    'num_attention_heads': num_attention_heads,
    'batch_size': batch_size,
    'learning rate': lr,
    'seed': seed
}
run_name = dataset_name+'i'+str(steps)+'f'+str(look_ahead)+'H2shrt_'+model_name+'_'+str(num_hidden_layers)+'L'+str(num_attention_heads)+'H'+str(hidden_size)+'HD'+'_bs'+str(batch_size)+'lr'+str(lr)
run_name = run_name + '105Kgc_v' + str(seed)
run = wandb.init(
    project="1dCA_ai4math24",
    name = run_name,
    config=run_cfg
)

optimizer = torch.optim.AdamW(model.parameters(), lr)

losses = []

num_steps = 105_000  # Define the total number of training steps
evaluate_every = 100  # Frequency of evaluation
wndb_log = 10

model.train()

loop = tqdm(range(num_steps), desc='Training')
data_iter = cycle(dataloader)  # Create an infinite loop of dataloader

for step in loop:
    
    input_ids, target_ids = next(data_iter)
    input_ids, target_ids = input_ids.to(device), target_ids.to(device)

    outputs = model(input_ids, labels=target_ids)
    loss = outputs.loss

    optimizer.zero_grad()
    loss.backward()
    total_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
    optimizer.step()
    current_loss = loss.item()
    loop.set_description(f"Step {step + 1}/{num_steps} - Loss: {current_loss:.4f}")
    loop.set_postfix(loss=current_loss)

    if step % evaluate_every == 0 and step != 0:
        val_accuracy = evaluate_model(model, small_val_dataloader, device)
        print(f"\nEvaluation at Step {step}: Accuracy = {val_accuracy:.4f}", "loss:", loss.item())
        wandb.log({'step': step, 'val acc': val_accuracy})

    if step % wndb_log == 0:
        losses.append(loss.detach().item())      
        wandb.log({'step': step, 'loss': current_loss})

loop.close()
wandb.finish()

Training:   0%|          | 0/105000 [00:00<?, ?it/s]

Validation:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluation at Step 100: Accuracy = 0.6303 loss: 0.17606563866138458


Validation:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluation at Step 200: Accuracy = 0.6312 loss: 0.17643029987812042


...

Validation:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluation at Step 104800: Accuracy = 0.7099 loss: 0.14408858120441437


Validation:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluation at Step 104900: Accuracy = 0.7091 loss: 0.14462910592556


loss,▇▆▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████▇▇▇
step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val acc,▂▄▅▅▆▇▇▇▇▇████████▇█████████████▁▂▂▂▁▃▃▃
loss,0.14464
step,104990
val acc,0.70907


In [9]:
# Define the checkpoint content
checkpoint = {
    'model_state_dict': model.state_dict(),            # Model's state dictionary
    'optimizer_state_dict': optimizer.state_dict(),        # Optimizer's state dictionary
    'step': step,                                      # Current training step
    'losses': losses,                                  # Loss history
    'config': run_cfg,
    'seed': seed
}

# Define a checkpoint file path
checkpoint_path = run_name + ".pth"

# Save the checkpoint
torch.save(checkpoint, checkpoint_path)

In [10]:
# Define the ID of the sample you want to inspect
sample_id = 11  # Replace with the ID of the sample you're interested in

# Fetch the specific sample from the dataset
sample_input_ids, sample_target_ids = dataset[sample_id]
sample_input_ids, sample_target_ids = sample_input_ids.unsqueeze(0).to(device), sample_target_ids.unsqueeze(0).to(device)

# Set the model to evaluation mode and get output
model.eval()
with torch.no_grad():
    output = model(sample_input_ids)

# Decode the tokens to strings for readability
decoded_input = tokenizer.decode(sample_input_ids[0].tolist())
decoded_target = tokenizer.decode(sample_target_ids[0].tolist())
decoded_prediction = tokenizer.decode(torch.argmax(output.logits, dim=-1)[0].tolist())

# Print input, target, prediction, and logits
print("inputs:",len(sample_input_ids),sample_input_ids)
print("targets:",len(sample_target_ids),sample_target_ids)
print(len(output),output)
print("Input:", len(sample_input_ids[0].tolist()),decoded_input)
print("Target:", len(sample_target_ids[0].tolist()), decoded_target)
print("Prediction:", decoded_prediction)
print("Logits:", len(output.logits[0]), output.logits[0])

# Optionally, you can convert logits to probabilities for interpretation
probabilities = torch.softmax(output.logits[0], dim=-1)
print("Probabilities:", probabilities)


inputs: 1 tensor([[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 2, 1, 0, 1,
         1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 0,
         1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 1, 1, 1, 0, 1, 1, 0, 0,
         0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
         0, 0, 0, 0, 1, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
         1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
         0, 1, 2, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 2,
         1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 2, 1, 1, 0,
         1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 

In [8]:
def evaluate_model_LH(model, val_dataloader, device, horizon):
    model.eval()  # Set the model to evaluation mode
    val_loop = tqdm(val_dataloader, leave=False, desc="Validation")
    correct = [0] * horizon
    total = [0] * horizon
    accuracy = [0] * horizon

    with torch.no_grad():
        for input_ids, target_ids in val_loop:
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            outputs = model(input_ids)
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            for t in range(horizon):

                # Create a mask where target_ids is not 3 (ignoring padding)
                #valid_mask = target_ids[:, t] != 3

                # Apply the mask to filter out -100 values
                start_idx  = orbit_len + t * state_len
                valid_predictions = predictions[:, start_idx : start_idx + state_len ]
                valid_targets = target_ids[:, start_idx : start_idx + state_len]
            

                correct[t] += (valid_predictions == valid_targets).sum().item()
                total[t] += len(valid_targets)

    for t in range(horizon):
        accuracy[t] = correct[t] / total[t] / state_len  if total[t] > 0 else 0
        
    return accuracy, correct, total

In [21]:
run_name = '1dCA_r2s20T20i10f4H2shrt_BERT_10L8H512HD_bs400lr0.0001105Kgc_v2'
checkpoint_path = run_name + ".pth"
checkpoint = torch.load(checkpoint_path, map_location='cuda:0')
model.load_state_dict(checkpoint['model_state_dict'], strict=False)

model.to(device)

/tmp/ipykernel_4065/3340699064.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location='cuda:0')


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-9): 10 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_af

In [22]:
val_accuracy = evaluate_model_LH(model, val_dataloader, device,look_ahead)
print(val_accuracy)
#print(f"\nEvaluation at Step {step}: Validation accuracy = {val_accuracy:.10f}", "loss:", loss.item())

Validation:   0%|          | 0/125 [00:00<?, ?it/s]

([0.989144761904762, 0.9702285714285714, 0.9482990476190477, 0.9211533333333334], [1038602, 1018740, 995714, 967211], [50000, 50000, 50000, 50000])


In [23]:
test_accuracy = evaluate_model_LH(model, test_dataloader, device,look_ahead)
print(test_accuracy)
#print(f"\nEvaluation at Step {step}: Test Accuracy = {test_accuracy:.10f}", "loss:", loss.item())

Validation:   0%|          | 0/250 [00:00<?, ?it/s]

([0.9890085714285713, 0.9697285714285714, 0.948247619047619, 0.9212452380952382], [2076918, 2036430, 1991320, 1934615], [100000, 100000, 100000, 100000])


In [14]:
Evaluation at Step 49999: Accuracy = 0.999925 loss: 0.0001053252344718203

SyntaxError: invalid syntax (1662878917.py, line 1)

In [ ]:
orbit_len = state_len * steps

In [ ]:
0.9765709523809524